### Create an API session

In [12]:
import pandas as pd

from weaveapi.records import *
from weaveapi.options import *
from weaveapi.filter import *
from weaveapi.weaveh import *

WEAVE_CONFIG = "config/demo_ailineage_owner.config"
nodeApi, session = connect_weave_api(WEAVE_CONFIG)

data_collection = "vault"

token = "USDC"
decimals = 6

{"res":"ok","data":"pong 1690016278938"}


#### Check initial node owner balance

Bridge address: https://mumbai.polygonscan.com/address/0xB485e0813b567E1629B8083026307d4d9E2cDE0B

In [2]:
account = session.publicKey
reply = nodeApi.balance(session, account, "", token).get()
print("Custodial wallet", account, float(reply["data"]) / pow(10, decimals) if "data" in reply else 0)

Custodial wallet weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q 1000000.0


### Create tables to store prompts

In [3]:
personas = [
    "solidity_engineer",
    "paris_guide",
    "pregnancy_nurse",
    "startup_advisor",
    "crypto_trader",
    "ethical_data_consultant"
]

#### Prompt proposals

In [4]:
for persona in personas:
    print(persona)
    layout = { 
        "columns": { 
            "id": { "type": "LONG", "isIndexed": True, "isUnique": True, "isNullable": False },
            "ts": { "type": "LONG" },
            "pubkey": { "type": "STRING" },
            "sig": { "type": "STRING" },
            "ip": { "type": "STRING" },
            "roles": { "type": "STRING" },
            "text": { "type": "STRING" }
        }, 
        "idColumnIndex": 0,  # Autogenerates IDs
        "timestampColumnIndex": 1, # Fills the column automatically with the network time
        "ownerColumnIndex": 2, # Fills the pubkey column automatically with the public key of the writer
        "signatureColumnIndex": 3, # Fills the column with an EdDSA signature of the record hash
        "sourceIpColumnIndex": 4, # Fills the column with an EdDSA signature of the record hash
        "allowedRolesColumnIndex": 5, # Allowed readers, specified by the writer
        "isLocal": False,
        "applyReadTransformations": True
    }

    table = persona + "_proposals"
    nodeApi.dropTable(session, data_collection, table).get()
    reply = nodeApi.createTable(session, data_collection, table, CreateOptions(False, False, layout)).get()
    print(reply)

solidity_engineer
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'solidity_engineer_proposals'}}
paris_guide
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'paris_guide_proposals'}}
pregnancy_nurse
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'pregnancy_nurse_proposals'}}
startup_advisor
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'startup_advisor_proposals'}}
crypto_trader
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'ac

#### Accepted prompts

In [5]:
for persona in personas:
    print(persona)
    layout = { 
        "columns": { 
            "id": { "type": "LONG", "isIndexed": True, "isUnique": True, "isNullable": False },
            "ts": { "type": "LONG" },
            "pubkey": { "type": "STRING" },
            "sig": { "type": "STRING" },
            "ip": { "type": "STRING" },
            "roles": { "type": "STRING" },
            "source": { "type": "STRING" },
            "text": { "type": "STRING" }
        }, 
        "idColumnIndex": 0,  # Autogenerates IDs
        "timestampColumnIndex": 1, # Fills the column automatically with the network time
        "ownerColumnIndex": 2, # Fills the pubkey column automatically with the public key of the writer
        "signatureColumnIndex": 3, # Fills the column with an EdDSA signature of the record hash
        "sourceIpColumnIndex": 4, # Fills the column with an EdDSA signature of the record hash
        "allowedRolesColumnIndex": 5, # Allowed readers, specified by the writer
        "isLocal": False,
        "applyReadTransformations": True
    }

    table = persona + "_prompts"
    nodeApi.dropTable(session, data_collection, table).get()
    reply = nodeApi.createTable(session, data_collection, table, CreateOptions(False, False, layout)).get()
    print(reply)

solidity_engineer
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'solidity_engineer_prompts'}}
paris_guide
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'paris_guide_prompts'}}
pregnancy_nurse
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'pregnancy_nurse_prompts'}}
startup_advisor
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'startup_advisor_prompts'}}
crypto_trader
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 

#### Superprompts

In [6]:
for persona in personas:
    print(persona)
    layout = { 
        "columns": { 
            "id": { "type": "LONG", "isIndexed": True, "isUnique": True, "isNullable": False },
            "ts": { "type": "LONG" },
            "pubkey": { "type": "STRING" },
            "sig": { "type": "STRING" },
            "ip": { "type": "STRING" },
            "roles": { "type": "STRING" },
            "text": { "type": "STRING" }
        }, 
        "idColumnIndex": 0,  # Autogenerates IDs
        "timestampColumnIndex": 1, # Fills the column automatically with the network time
        "ownerColumnIndex": 2, # Fills the pubkey column automatically with the public key of the writer
        "signatureColumnIndex": 3, # Fills the column with an EdDSA signature of the record hash
        "sourceIpColumnIndex": 4, # Fills the column with an EdDSA signature of the record hash
        "allowedRolesColumnIndex": 5, # Allowed readers, specified by the writer
        "isLocal": False,
        "applyReadTransformations": True
    }

    table = persona + "_superprompts"
    nodeApi.dropTable(session, data_collection, table).get()
    reply = nodeApi.createTable(session, data_collection, table, CreateOptions(False, False, layout)).get()
    print(reply)

solidity_engineer
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'solidity_engineer_superprompts'}}
paris_guide
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'paris_guide_superprompts'}}
pregnancy_nurse
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'pregnancy_nurse_superprompts'}}
startup_advisor
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'startup_advisor_superprompts'}}
crypto_trader
{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'wea

#### Write some sample prompt

In [7]:
table = "paris_guide_proposals"

text = "You are a well trained Paris City guide that knows every attraction. You have to give a recommendation for a tourist such that it is both funny and interesting, based on the following input: "

records = Records(table, [
    [ None, None, None, None, None, None, text ]
])
reply = nodeApi.write(session, data_collection, records, WRITE_DEFAULT).get()
print(reply)


text = "As a Paris City Guide recommend me something to do "
records = Records(table, [
    [ None, None, None, None, None, None, text ]
])
reply = nodeApi.write(session, data_collection, records, WRITE_DEFAULT).get()
print(reply)

text = "As a Paris City Guide for foreigners you have to recommend the best place to visit in order to meet interesting people. What would that be, if the visitor wants "
records = Records(table, [
    [ None, None, None, None, None, None, text ]
])
reply = nodeApi.write(session, data_collection, records, WRITE_DEFAULT).get()
print(reply)

{'res': 'ok', 'target': {'operationType': 'WRITE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'paris_guide_proposals'}, 'data': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q,rbmtntkzhI54Zu2icINzmN+vFIMhr6tK0ljUnq9hHaY=,2fY2dFSKKPNKzhkNCvdfjNdVGW9KH8TBCK15zTTKf2XfZ1GimdxPSpUmBgpco3annVcFCYW84YvQdiWiybFGLkkF', 'ids': '1'}
{'res': 'ok', 'target': {'operationType': 'WRITE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'paris_guide_proposals'}, 'data': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q,KLZkMkd8JyVGwOyhDCH7W6Mx71QhUj6pBuqjhJ4ZEsk=,3fsexeQ6aXQPfMGE6DhGG1WNdbcooVHyxFudoLz7Q11ws7AAPxnbt5kABcrXdNpp3Tn5vneP4ukEReRHvZDVwHex', 'ids': '2'}
{'res': 'ok', 'target': {'operationType': 'WRITE', 'organization': 'weavedemo', 'account': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q', 'scope': 'vault', 'table': 'paris_guide_pro

#### Test read proposed prompts

In [8]:
table = "paris_guide_proposals"

reply = nodeApi.read(session, data_collection, table, None, READ_DEFAULT_NO_CHAIN).get()
#print(reply)
df = pd.DataFrame(reply["data"])
display(df.tail())

sample_prompt = df.iloc[0]["text"]

,id,ts,pubkey,sig,ip,text
0,1,1690007518671,weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioS...,"[{""sig"":{""recordsHash"":""rbmtntkzhI54Zu2icINzmN...",172.174.28.163,You are a well trained Paris City guide that k...
1,2,1690007518725,weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioS...,"[{""sig"":{""recordsHash"":""KLZkMkd8JyVGwOyhDCH7W6...",172.174.28.163,As a Paris City Guide recommend me something t...
2,3,1690007518774,weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioS...,"[{""sig"":{""recordsHash"":""CzriI5SNDKnG1qvUOak8U2...",172.174.28.163,As a Paris City Guide for foreigners you have ...


#### Records have hashes and signatures from the source

In [9]:
display(json.loads(df.iloc[-1]["sig"]))

[{'sig': {'recordsHash': 'CzriI5SNDKnG1qvUOak8U2ClHg8a4skXp1EcP6hpbis=',
   'count': '3',
   'pubKey': 'weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioSu38Q',
   'prevRecordsHash': 'KLZkMkd8JyVGwOyhDCH7W6Mx71QhUj6pBuqjhJ4ZEsk=',
   'sig': '5EWTgwvyrag6v9bYhSvWNA6tLjXB1B1PbZnhn858gGEY1FukosYKMW8Q28GQWjcZnqxn1Ys5v4gLmUPWQtuZ6nYY'}}]

### Run compute task to accept prompts

In [ ]:
timeoutSec = 300
params = {
    "persona": "paris_guide",
    "accept": [ 1, 3]
}
options = ComputeOptions(False, timeoutSec, 0, None, params)

### TODO. Leave auth for later
reply = nodeApi.compute(session, "gcr.io/weavechain/approve_prompts", options).get()
display(reply)

#### Test read accepted prompts

In [14]:
table = "paris_guide_prompts"

reply = nodeApi.read(session, data_collection, table, None, READ_DEFAULT_NO_CHAIN).get()
#print(reply)
df = pd.DataFrame(reply["data"])
display(df.tail())

sample_prompt = df.iloc[0]["text"]

,id,ts,pubkey,sig,ip,roles,source,text
0,1,1690016205896,weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioS...,"[{""sig"":{""recordsHash"":""FkVUQ7UnnZ8x2Nz21qJs1t...",89.137.100.219,*,weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioS...,You are a well trained Paris City guide that k...
1,2,1690016206880,weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioS...,"[{""sig"":{""recordsHash"":""IbLKkVDQ96sV/EYgUXRvgA...",89.137.100.219,*,weave29LwVcvu5rrg9bfbp7UhQ89unqStMzQvtn7Td5ioS...,As a Paris City Guide for foreigners you have ...


### Run compute task to generate a superprompt

In [ ]:
timeoutSec = 300
params = {
    "persona": "paris_guide"
}
options = ComputeOptions(False, timeoutSec, 0, None, params)

reply = nodeApi.compute(session, "gcr.io/weavechain/???", options).get()
display(reply)

#### Compute lineage

In [ ]:
task_reply = reply["data"]

print("Compute Task Hash:", task_reply["computeHash"])
print("Input Hash:", task_reply["inputHash"])
print("Root Hashes:", task_reply.get("rootHashes"))
print("Output Hash:", task_reply["outputHash"])
print("Output Signature:", task_reply["outputSignatureTs"])

signature = task_reply["outputSignature"]
match = nodeApi.verifyLineageSignature(signature, task_reply.get("inputHash"), task_reply.get("computeHash"), task_reply.get("paramsHash"), task_reply["output"])
print("\nSignature Match:", match)a

### TODO Check balance for writers (and use different accounts)

In [ ]:
account = sample["pubkey"]
reply = nodeApi.balance(session, account, "", token).get()
print("Custodial wallet:", session.publicKey, float(reply["data"]) / pow(10, decimals) if "data" in reply else 0)

### Blockchain anchors for immutability proofs (for Paris guide, accepted prompts)

#### Records have immutability proofs on blockchain

https://mumbai.polygonscan.com/address/0x5FFD5EB6B8742e38884Ce33480e09173B830613B

In [15]:
reply = nodeApi.hashes(session, data_collection, table, None, READ_DEFAULT_NO_CHAIN).get()
#print(reply)
dh = pd.DataFrame([ [x, reply["data"][x]] for x in list(reply["data"]) ], columns=["ID", "Hash"])
display(dh)

,ID,Hash
0,1,aiZx6BPASuu2toOygvV9fnBLYEmQ7WtfTA238zPLPAc=


#### Merkle Tree Proofs

#### Records have a different type of immutability proof offering the ability to check inclusion via merkle trees

https://mumbai.polygonscan.com/address/0x32D8c718Cd46645fd8D46CCbeB40137b54458F9e

In [16]:
from binarytree import Node

def showtree(tree):
    prev = None
    root = None
    lvl = 1
    for l in tree.split(";"):
        level = l.split(",")
        print(lvl, level)
        lvl += 1
        nodes = []
        for i in range(len(level)):
            pidx = int(i / 2)
            node = Node(level[i][:3] + ".." + level[i][-3:])
            if root is None:
                root = node
            nodes.append(node)
            if prev is not None:
                parent = prev[pidx]
                if parent.left is None:
                    parent.left = node
                else:
                    parent.right = node
        prev = nodes
    
    print(root)

salt = "salt1234" # Same salt used for records hashes, this can be improved to have different salts for *each distinct writer*
digest = "SHA-256"

filter = Filter(None, None, None, None, [ "text" ])
reply = nodeApi.merkleTree(session, data_collection, table, filter, salt, digest, READ_DEFAULT_NO_CHAIN).get()
tree = reply["data"]["tree"]
rootHash = reply["data"]["rootHash"]
ts = reply["data"]["timestamp"]
rootHashSignature = reply["data"]["signature"]

print("Generated at", ts)
print("Root Hash", rootHash)
print("Signature", rootHashSignature)
print("")
showtree(tree)
# We've built a Merkle Tree at a specific time, signed by the node that created it.
# The Merkle Leaves are salted hashes of the data in the table. Binance did an enumeration of coins.

Generated at 1690016342230
Root Hash DuisniJjpv8vpkRhAkiFHZw42KPhi36XRP83xUAoUzmL
Signature 3KvQsX2eSsg89kuYUAezDA3YzatcA5hMaMskNRAYWa1b3p3d5Vy3ejnzu8R8271wbALk2d8ZCb3pJmTHQHw9ndPz

1 ['DuisniJjpv8vpkRhAkiFHZw42KPhi36XRP83xUAoUzmL']
2 ['H6UdXXMF71En5drhGTEhUDkACdxSeEpLewXxbbUicRdz', 'DnVaHn9sMpgeDyrLdqnhVSjj7mNDksJNP6eMpNE7x2yC']

     ____Dui..zmL___
    /               \
H6U..Rdz          DnV..2yC



#### Check the root hash stored on chain

In [17]:
table = "paris_guide_prompts"

reply = nodeApi.rootHash(session, data_collection, table).get()
#print(reply)
data = reply["data"]
display(data)

chainRootHash = data["rootHash"]
print("\nMatching:", rootHash == chainRootHash) # data hashes are salted. The salt needs to match what's configured on the network to have a match

{'signature': '3skp9PPZH7Sdt7XkponQGmCY1g4678VsXXrWz5Ma3TuFX5yoxMomVW5xU8Hh5eVUUdYH8BfeXZCmk4pK3RnU8HDm',
 'rootHash': 'DuisniJjpv8vpkRhAkiFHZw42KPhi36XRP83xUAoUzmL',
 'timestamp': '1690016346567'}


Matching: True


#### Verify root hash signature

In [18]:
signingKey = nodeApi.sigKey().get()["data"]
print("Node Public Key:", signingKey)

Node Public Key: GfHq2tTVk9z4eXgyXVRvnn4hnpsfgFCfen9n1KoqnLp9ANhnmj9BsdJvv9Gg


In [20]:
toSign = rootHash + " " + ts
#print(rootHash)
check = nodeApi.verifySignature(rootHashSignature, toSign)
print("Check signature from merkleTree() call:", check)

toSign = data["rootHash"] + " " + data["timestamp"]
check = nodeApi.verifySignature(data["signature"], toSign)
print("Check signature from blockchain:", check)

# Note, verifySignature isn't unique to Weavechain, it's EdDSA public key cryptography and the verification is done locally

Check signature from merkleTree() call: True
Check signature from blockchain: True


### Verify inclusion

#### With SHA-256 hash

Compute the hash of one of the articles

In [21]:
row = [ sample_prompt ]
recordHash = nodeApi.hashRecord(row, salt)
print(recordHash)

H6UdXXMF71En5drhGTEhUDkACdxSeEpLewXxbbUicRdz


Check the hash presence and validate the Merkle Tree

In [22]:
reply = nodeApi.verifyMerkleHash(session, tree, recordHash).get()
print(reply["data"])

true


Or get a Merkle Proof for the record hash and verify it

In [23]:
table = "paris_guide_prompts"

reply = nodeApi.merkleProof(session, data_collection, table, recordHash).get()
print(reply)
data = reply["data"]
display(data)

hashes = [ h.split(",") for h in data["proof"].split(";") ]
display(hashes)

proofSignature = data["signature"]
toSign = data["proof"] + " " + data["timestamp"] # the node signed the proof and the timestamp
check = nodeApi.verifySignature(proofSignature, toSign)
print(check)

{'res': 'ok', 'data': {'signature': '5mm4t3sNJSD8ktCtVhJxHnDqbj7qRZdrxc4Ti1bDwm6yuTwm6Y7TwH4GQNt2WPK2t8QdygkS33TszpPurAoCCofH', 'rootHash': 'DuisniJjpv8vpkRhAkiFHZw42KPhi36XRP83xUAoUzmL', 'proof': 'H6UdXXMF71En5drhGTEhUDkACdxSeEpLewXxbbUicRdz,DnVaHn9sMpgeDyrLdqnhVSjj7mNDksJNP6eMpNE7x2yC', 'timestamp': '1690016371868'}}


{'signature': '5mm4t3sNJSD8ktCtVhJxHnDqbj7qRZdrxc4Ti1bDwm6yuTwm6Y7TwH4GQNt2WPK2t8QdygkS33TszpPurAoCCofH',
 'rootHash': 'DuisniJjpv8vpkRhAkiFHZw42KPhi36XRP83xUAoUzmL',
 'proof': 'H6UdXXMF71En5drhGTEhUDkACdxSeEpLewXxbbUicRdz,DnVaHn9sMpgeDyrLdqnhVSjj7mNDksJNP6eMpNE7x2yC',
 'timestamp': '1690016371868'}

[['H6UdXXMF71En5drhGTEhUDkACdxSeEpLewXxbbUicRdz',
  'DnVaHn9sMpgeDyrLdqnhVSjj7mNDksJNP6eMpNE7x2yC']]

True


### Use persona

In [ ]:
timeoutSec = 300
params = {
    "persona": "paris_guide",
    "prompt": "Please tell me where to spend my Sunday morning "
}
options = ComputeOptions(False, timeoutSec, 0, None, params)

reply = nodeApi.compute(session, "gcr.io/weavechain/???", options).get()
display(reply)